```{eval-rst}
.. currentmodule:: xarray.indexes
```

# Intervals with `CFIntervalIndex`

```{seealso}
Learn more at the [Climate and Forecast Conventions on bounds variables](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.12/cf-conventions.html#cell-boundaries).
```

```{warning}
This Index is [in development](https://github.com/pydata/xarray/pull/10296), and is not available in the released Xarray version yet.
```

## Highlights

1. {py:class}`CFIntervalIndex`, models intervals using _two_ arrays, one 2D array represents interval bounds, and a second 1D array represents the "central values". Such a format for recording "cell bounds" is recommended by the [CF conventions](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.12/cf-conventions.html#cell-boundaries).
1. {py:class}`CFIntervalIndex` overrides Xarray's default coordinate propagation rules using {py:func}`xarray.Index.should_add_coord_to_array`. By default, when extracting a DataArray from a Dataset, Xarray will only include those coordinate variables whose dimensions are a subset of the dimensions of the extracted DataArray.

## Example

We will load the standard air temperature dataset, but add a new `time_bounds` variable.

In [1]:
%xmode minimal

import pandas as pd
import xarray as xr

xr.set_options(display_expand_indexes=True, display_expand_attrs=False)
pd.set_option('display.max_seq_items', 10)

orig = xr.tutorial.open_dataset("air_temperature")
orig

Exception reporting mode: Minimal


<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB ...
Attributes: (5)

Let's replace the `time` vector with an IntervalIndex, assuming that the data represent averages over 6 hour periods centered at 00h, 06h, 12h, 18h

In [2]:
left = orig.time - pd.Timedelta("3h")
right = orig.time + pd.Timedelta("3h")
time_bounds = xr.concat([left, right], dim="bounds")
time_bounds

<xarray.DataArray 'time' (bounds: 2, time: 2920)> Size: 47kB
array([['2012-12-31T21:00:00.000000000', '2013-01-01T03:00:00.000000000',
        '2013-01-01T09:00:00.000000000', ...,
        '2014-12-31T03:00:00.000000000', '2014-12-31T09:00:00.000000000',
        '2014-12-31T15:00:00.000000000'],
       ['2013-01-01T03:00:00.000000000', '2013-01-01T09:00:00.000000000',
        '2013-01-01T15:00:00.000000000', ...,
        '2014-12-31T09:00:00.000000000', '2014-12-31T15:00:00.000000000',
        '2014-12-31T21:00:00.000000000']],
      shape=(2, 2920), dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Dimensions without coordinates: bounds

In [3]:
orig.coords["time_bounds"] = time_bounds
orig.time.attrs["bounds"] = "time_bounds"  # add the attribute for CF compliance
orig

<xarray.Dataset> Size: 31MB
Dimensions:      (lat: 25, time: 2920, lon: 53, bounds: 2)
Coordinates:
  * lat          (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon          (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * time         (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
    time_bounds  (bounds, time) datetime64[ns] 47kB 2012-12-31T21:00:00 ... 2...
Dimensions without coordinates: bounds
Data variables:
    air          (time, lat, lon) float64 31MB ...
Attributes: (5)

### Assigning

We will drop the existing PandasIndex along `"time"` and add a new {py:class}`CFIntervalIndex`

In [4]:
from xarray.indexes import CFIntervalIndex

indexed = orig.drop_indexes("time").set_xindex(
    ["time", "time_bounds"], CFIntervalIndex, closed="left"
)
indexed

<xarray.Dataset> Size: 31MB
Dimensions:      (lat: 25, time: 2920, lon: 53, bounds: 2)
Coordinates:
  * lat          (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon          (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * time         (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
  * time_bounds  (time, bounds) datetime64[ns] 23kB 2012-12-31T21:00:00 ... 2...
Dimensions without coordinates: bounds
Data variables:
    air          (time, lat, lon) float64 31MB ...
Indexes:
  ┌ time         CFIntervalIndex
  └ time_bounds
Attributes: (5)

### Coordinate variable propagation

A classic issue with Xarray is that bounds variables don't get propagated by default.

Note that the `"time_bounds"` variable is lost when pulling out the `"air"` DataArray.
Important information has been lost!

In [5]:
orig["air"]

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)> Size: 31MB
[3869000 values with dtype=float64]
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Attributes: (11)

But {py:class}`CFIntervalIndex` overrides that rule, and can propagate the `"time_bounds"` variable since it is required to correctly propagate the Index.

In [6]:
indexed["air"]

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)> Size: 31MB
[3869000 values with dtype=float64]
Coordinates:
  * lat          (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon          (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * time         (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
  * time_bounds  (time, bounds) datetime64[ns] 23kB 2012-12-31T21:00:00 ... 2...
Indexes:
  ┌ time         CFIntervalIndex
  └ time_bounds
Attributes: (11)

### Indexing

Index the intervals using the `'time'` coordinate

In [7]:
indexed.sel(time=["2013-01-02 10:00", "2013-01-04 10:00"])

<xarray.Dataset> Size: 22kB
Dimensions:      (lat: 25, time: 2, lon: 53, bounds: 2)
Coordinates:
  * lat          (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon          (lon) float32 212B 200.0 202.5 205.0 ... 325.0 327.5 330.0
  * time         (time) datetime64[ns] 16B 2013-01-02T12:00:00 2013-01-04T12:...
  * time_bounds  (time, bounds) datetime64[ns] 16B 2013-01-02T09:00:00 ... 20...
Dimensions without coordinates: bounds
Data variables:
    air          (time, lat, lon) float64 21kB ...
Indexes:
  ┌ time         CFIntervalIndex
  └ time_bounds
Attributes: (5)